# Итоговая аттестация аналитик данных 

## 1. Исследовательский анализ данных 

### 1.1. Предварительный обзор и предобработка давнных

Набор данных, содержащий информацию о фильмах и сериалах всем известного интернет-кинотеатра Netflix

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import levene

In [2]:
# Загрузка данных
df = pd.read_csv('data/netflix1.csv', skipinitialspace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 686.8+ KB


In [3]:
# Проверка на дубликаты
print(f'В датасете {len(df) - len(df.drop_duplicates())} дубликатов строк')
# Проверка на пропущенные значения
df.isnull().sum()

В датасете 0 дубликатов строк


show_id         0
type            0
title           0
director        0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
dtype: int64

В наборе данных нет дублирующий строк и пропущенных значений

In [4]:
#Первый взляд на дынные
df.head(5)

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"


In [5]:
#посмотрим какие значения может принимать type
df.type.value_counts()

type
Movie      6126
TV Show    2664
Name: count, dtype: int64

In [6]:
#посмотрим какие значения может принимать rating
df.rating.value_counts()

rating
TV-MA       3205
TV-14       2157
TV-PG        861
R            799
PG-13        490
TV-Y7        333
TV-Y         306
PG           287
TV-G         220
NR            79
G             41
TV-Y7-FV       6
NC-17          3
UR             3
Name: count, dtype: int64

In [9]:
#посмотрим какие значения может принимать country
df.country.value_counts()

country
United States     3240
India             1057
United Kingdom     638
Pakistan           421
Not Given          287
                  ... 
Iran                 1
West Germany         1
Greece               1
Zimbabwe             1
Soviet Union         1
Name: count, Length: 86, dtype: int64

In [8]:
#посмотрим какие значения может принимать duration
df.duration.value_counts()

duration
1 Season      1791
2 Seasons      421
3 Seasons      198
90 min         152
97 min         146
              ... 
5 min            1
16 min           1
186 min          1
193 min          1
11 Seasons       1
Name: count, Length: 220, dtype: int64

In [10]:
#посмотрим  duration в разрезе типа контента
df_movies = df[df.type == 'Movie'] 
df_tv_show = df[df.type == 'TV Show'] 
print(df_movies.duration.value_counts())
df_tv_show.duration.value_counts()

duration
90 min     152
93 min     146
94 min     146
97 min     146
91 min     144
          ... 
208 min      1
5 min        1
16 min       1
186 min      1
312 min      1
Name: count, Length: 205, dtype: int64


duration
1 Season      1791
2 Seasons      421
3 Seasons      198
4 Seasons       94
5 Seasons       64
6 Seasons       33
7 Seasons       23
8 Seasons       17
9 Seasons        9
10 Seasons       6
15 Seasons       2
13 Seasons       2
12 Seasons       2
17 Seasons       1
11 Seasons       1
Name: count, dtype: int64

In [11]:
#Проверю может ли продолжительность фильм измерятся в сезонах
df_movies.duration.str.contains('Season').sum()

0

In [12]:
#Создадим множество для типов жанров
genre_ = set()
for i in range(df.shape[0]):
    list_ = df['listed_in'][i].split(',') # Создаю список из строки
    set_ = {list_[n].strip() for n in range(len(list_))} #добавляю элемент без начальных пробелов списка в множество
    genre_.update(set_) 

print(f'В каталоге содержится {len(genre_)} жанров фильмов и телешоу:')
genre_


В каталоге содержится 42 жанров фильмов и телешоу:


{'Action & Adventure',
 'Anime Features',
 'Anime Series',
 'British TV Shows',
 'Children & Family Movies',
 'Classic & Cult TV',
 'Classic Movies',
 'Comedies',
 'Crime TV Shows',
 'Cult Movies',
 'Documentaries',
 'Docuseries',
 'Dramas',
 'Faith & Spirituality',
 'Horror Movies',
 'Independent Movies',
 'International Movies',
 'International TV Shows',
 "Kids' TV",
 'Korean TV Shows',
 'LGBTQ Movies',
 'Movies',
 'Music & Musicals',
 'Reality TV',
 'Romantic Movies',
 'Romantic TV Shows',
 'Sci-Fi & Fantasy',
 'Science & Nature TV',
 'Spanish-Language TV Shows',
 'Sports Movies',
 'Stand-Up Comedy',
 'Stand-Up Comedy & Talk Shows',
 'TV Action & Adventure',
 'TV Comedies',
 'TV Dramas',
 'TV Horror',
 'TV Mysteries',
 'TV Sci-Fi & Fantasy',
 'TV Shows',
 'TV Thrillers',
 'Teen TV Shows',
 'Thrillers'}

Набор данных содержит 8790 строк и 10 столбцов со следующей инфомацией:
- show_id - уникальный идетификатор фильма (телешоу) в формате 'sчисло' (тип данных - object)
- type - тип контента (тип данных - object). Может примимать значения:
    - Movie - фильм;
    - TV Show - телешоу.
- title - название фильма (телешоу) (тип данных - object)
- director - режиссер (тип данных - object)
- country - страна происхождения(тип данных - object) 
    - страна происхождения известна не для всех фильмов, есть значение **Not Given**
- date_added - дата добавления в каталог Netflix (тип данных - object)
- release_year - год выхода в прокат фильма (телешоу) (тип данных - int64)
- rating - возрастное ограничение фильма (телешоу) - (тип данных - object).  Может примимать значения:
    - TV-MA — Эта программа предназначена для просмотра взрослыми и поэтому может не подходить для детей младше 17 лет;
    - TV-Y — Эта программа предназначена для очень юной аудитории, в том числе для детей от 2 до 6 лет;
    - TV-Y7 — Эта программа предназначена для детей от 7 лет и старше;
    - TV-Y7-FV — Эта программа демонстрируют больше «фантастического насилия», которое может быть более интенсивным или боевым, чем другие программы с рейтингом TV-Y7;
    - TV-G обычно подходят для всех возрастов;
    - TV-PG — Рекомендуется родительское руководство; эти программы могут быть неподходящими для детей младшего возраста.
    - PG-13 — Детям до 13 лет просмотр не желателен;
    - PG – кино разрешённое для просмотра всем, но маленьким детям рекомендуется просмотр с родителями;
    - NC-17 – зрители до 18 не допускаются (в названии «17» потому что включительно);
    - R – зрители до 17 лет должны присутствовать в зале с сопровождением родителей;
    - G – кино без всяких возрастных ограничений;
    - NR - нет рейтинга;
    - UR - фильм вышедший в прокат до введения системы рейтингов, то есть до 1 ноября 1968 года.
- duration - продолжительность  (тип данных - object).  Может примимать значения:
    - для фильма в минутах;
    - для телешою в сезонах.
- listed_in - жанры фильма (телешоу) (тип данных - object). Может примимать значения:
    - список из жанров фильма (всего в каталоге 42 жанра)


Для удобства провдения исследовательского анализа набора данных:
- поменяю тип данных **date_added** 
- создам столбец **duration_temp** с типом данных **int64** с продолжителностью в минутах для фильмов и количеством сезонов для телешоу

In [13]:
#Меняю тип данных в date_added
df.date_added = df.date_added.astype("datetime64[ns]")
#Вставляю  столбец
df.insert(9, 'duration_temp', 0 , allow_duplicates = False)
#Заполняю столбец минутами для фильмов и количеством сезонов для телешоу
for i in range(df.shape[0]):
    list_ = df.duration[i].split(' ')
    df.loc[i,'duration_temp'] = int(list_[0])

### 1.2. Исследовательский анализ данных (EDA)